# Performance Tuning: SQL & Table Optimization

**The Situation:** Leadership wants dashboards, predictive models, and AI agents ready by Friday. Your plane IoT data is growing fast, and queries that worked yesterday are timing out today.

**The Problem:** Slow queries = missed deadlines + angry leadership

**The Solution:** Get familiar with both SQL and table optimization techniques to get sub-second query times.

---

## What You'll Learn (30 minutes)

✅ **SQL Optimization:** Predicate pushdown, join strategies, broadcast hints  
✅ **Table Optimization:** File compaction, Z-Ordering, Liquid Clustering  
✅ **Materialized Views:** Pre-compute expensive aggregations  
✅ **Deletion Vectors:** Fast updates and deletes  
✅ **Performance Measurement:** Before/after comparisons  

---

## Prerequisites

- Completed Day 1 & 2
- `sensor_bronze`, `dim_factories`, `dim_devices` tables loaded
- SQL Warehouse or cluster running

---

**References:**
- [Delta Lake Performance](https://docs.databricks.com/en/delta/tune-file-layout.html)
- [Liquid Clustering](https://docs.databricks.com/en/delta/clustering.html)
- [Query Optimization](https://docs.databricks.com/en/optimizations/)


In [0]:
# Configuration
CATALOG = 'main'
SCHEMA = 'onboarding'
USER = spark.sql("SELECT current_user()").collect()[0][0].split('@')[0].replace('.', '_')

print(f"Using: {CATALOG}.{SCHEMA}")
print(f"User: {USER}")


## Part 1: Understanding Performance Bottlenecks

### Common Performance Killers

| Problem | Impact | Solution |
|---------|--------|----------|
| 🐌 **Small Files** | Too many file opens | OPTIMIZE |
| 🐌 **Full Table Scans** | Read entire table | Z-Order, predicates |
| 🐌 **Data Shuffle** | Network overhead | Broadcast joins |
| 🐌 **Wrong Join Type** | Memory spills | Join hints |
| 🐌 **Repeated Computation** | Wasted resources | Materialized views |
| 🐌 **Inefficient Predicates** | No pushdown | Proper filters |

### Performance Toolkit

**SQL Optimization:**
- Predicate pushdown (filter early)
- Join hints (BROADCAST, SHUFFLE_HASH)
- Proper WHERE clause design

**Table Optimization:**
- File compaction (OPTIMIZE)
- Z-Ordering (data layout)
- Liquid Clustering (auto-optimize)
- Deletion Vectors (fast updates)

**Query Results:**
- Materialized Views
- Caching


## Part 2: Creating a "Bad" Table for Demonstration

Let's intentionally create a poorly optimized table with:
- Many small files (simulating streaming ingestion)
- Random data layout (no locality)
- No optimization

This represents what happens in real production systems without proper maintenance!


In [0]:
# Step 1: Create unoptimized table with random layout
spark.sql(f"DROP TABLE IF EXISTS {CATALOG}.{SCHEMA}.{USER}_sensor_unoptimized")

spark.sql(f"""
CREATE TABLE {CATALOG}.{SCHEMA}.{USER}_sensor_unoptimized
AS
SELECT 
    device_id,
    trip_id,
    factory_id,
    model_id,
    timestamp,
    airflow_rate,
    rotation_speed,
    air_pressure,
    temperature,
    delay,
    density
FROM {CATALOG}.{SCHEMA}.{USER}_sensor_bronze
ORDER BY RAND()  -- Random order = worst case for data locality!
LIMIT 200000  -- Use subset for demo
""")

print("✅ Created unoptimized table with random layout")


In [0]:
# Step 2: Simulate many small files (like streaming writes)
# This is what happens with continuous ingestion without auto-compaction

for i in range(15):  # Create 15 small file batches
    spark.sql(f"""
    INSERT INTO {CATALOG}.{SCHEMA}.{USER}_sensor_unoptimized
    SELECT 
        device_id,
        trip_id,
        factory_id,
        model_id,
        timestamp,
        airflow_rate,
        rotation_speed,
        air_pressure,
        temperature,
        delay,
        density
    FROM {CATALOG}.{SCHEMA}.{USER}_sensor_bronze
    WHERE MOD(device_id, 15) = {i}
    LIMIT 800
    """)

print("✅ Created many small files (simulating poor ingestion patterns)")


In [0]:
# Check table statistics - look at the file count!
display(spark.sql(f"""
DESCRIBE DETAIL {CATALOG}.{SCHEMA}.{USER}_sensor_unoptimized
""").select("numFiles", "sizeInBytes", "minReaderVersion", "minWriterVersion"))


### 🔍 What to Look For:

- **numFiles**: High number (hundreds of thousands or millions) = Performance problem!
- **sizeInBytes**: Total size, but spread across too many files

**Problem:** Every query must:
1. List all files
2. Open each file
3. Read metadata
4. Scan for relevant data

With many small files, overhead dominates actual work!


## Part 3: SQL Optimization - Predicate Pushdown

**Key Concept:** Push filters as close to the data as possible.

### ❌ Bad: Function on Column (Prevents Pushdown)


In [0]:
import time

# BAD: Using SUBSTRING on timestamp prevents predicate pushdown
start = time.time()

result_bad = spark.sql(f"""
SELECT 
    device_id,
    factory_id,
    AVG(temperature) as avg_temp
FROM {CATALOG}.{SCHEMA}.{USER}_sensor_unoptimized
WHERE SUBSTRING(CAST(timestamp AS STRING), 1, 10) >= DATE_SUB(CURRENT_DATE(), 7)
GROUP BY device_id, factory_id
""")

count_bad = result_bad.count()
time_bad = time.time() - start

print(f"❌ BAD Query Time: {time_bad:.2f} seconds")
print(f"   Results: {count_bad} rows")
print(f"   Problem: Function on column prevents statistics-based filtering!")


# GOOD: Direct filter on timestamp column enables predicate pushdown



In [0]:
start = time.time()

result_good = spark.sql(f"""
SELECT 
    device_id,
    factory_id,
    AVG(temperature) as avg_temp
FROM {CATALOG}.{SCHEMA}.{USER}_sensor_unoptimized
WHERE timestamp >= CURRENT_DATE() - INTERVAL 7 DAYS
GROUP BY device_id, factory_id
""")

count_good = result_good.count()
time_good = time.time() - start

print(f"✅ GOOD Query Time: {time_good:.2f} seconds")
print(f"   Results: {count_good} rows")
print(f"   Speedup: {time_bad/time_good:.1f}x faster!")
print(f"   Reason: Databricks can use file statistics to skip irrelevant files")

### 💡 Predicate Pushdown Best Practices

**DO:**
```sql
WHERE timestamp >= '2024-01-01'  -- Direct column comparison
WHERE device_id IN (1, 2, 3)     -- Direct value check
WHERE factory_id = 'A06'          -- Equality on column
```

**DON'T:**
```sql
WHERE DATE(timestamp) = '2024-01-01'      -- Function prevents pushdown
WHERE SUBSTRING(device_id, 1, 2) = '10'   -- Function on column
WHERE UPPER(factory_id) = 'A06'           -- Transformation blocks optimization
```


## Part 4: SQL Optimization - Join Strategies

### Understanding Join Types

| Join Type | Best For | Cost |
|-----------|----------|------|
| **Broadcast Join** | Small table (< 10MB) | Low - no shuffle |
| **Shuffle Hash Join** | Large tables | High - shuffle both |
| **Sort Merge Join** | Large sorted tables | Medium |

### ❌ Bad: Let Spark guess (might shuffle large tables)


# Without hint - Spark might choose inefficient join strategy


In [0]:
start = time.time()

result_no_hint = spark.sql(f"""
SELECT 
    s.device_id,
    f.factory_name,
    f.region,
    COUNT(*) as reading_count,
    AVG(s.temperature) as avg_temp
FROM {CATALOG}.{SCHEMA}.{USER}_sensor_unoptimized s
JOIN {CATALOG}.{SCHEMA}.{USER}_dim_factories f 
  ON s.factory_id = f.factory_id
GROUP BY s.device_id, f.factory_name, f.region
""")

count_no_hint = result_no_hint.count()
time_no_hint = time.time() - start

print(f"⚠️  No Hint Query Time: {time_no_hint:.2f} seconds")
print(f"   Spark may shuffle both tables unnecessarily")


### ✅ Good: Broadcast Small Dimension Table


In [0]:
# With BROADCAST hint - force efficient join strategy
start = time.time()

result_broadcast = spark.sql(f"""
SELECT 
    /*+ BROADCAST(f) */
    s.device_id,
    f.factory_name,
    f.region,
    COUNT(*) as reading_count,
    AVG(s.temperature) as avg_temp
FROM {CATALOG}.{SCHEMA}.{USER}_sensor_unoptimized s
JOIN {CATALOG}.{SCHEMA}.{USER}_dim_factories f 
  ON s.factory_id = f.factory_id
GROUP BY s.device_id, f.factory_name, f.region
""")

count_broadcast = result_broadcast.count()
time_broadcast = time.time() - start

print(f"✅ Broadcast Join Time: {time_broadcast:.2f} seconds")
print(f"   Speedup: {time_no_hint/time_broadcast:.1f}x faster!")
print(f"   Only dimension table sent to executors - no shuffle of fact table!")


### 💡 Join Optimization Rules

**BROADCAST when:**
- Dimension table < 10MB
- Reference data (factories, models, devices)
- Lookup tables

**Let Spark choose when:**
- Both tables are large
- Join cardinality is unknown
- Adaptive Query Execution is enabled (default)


## Part 5: Table Optimization - File Compaction

**Problem:** Many small files cause overhead

**Solution:** OPTIMIZE command compacts small files into larger ones

**Target:** 128MB - 1GB per file (default: 1GB)


In [0]:
# Check current file situation
detail_before = spark.sql(f"""
DESCRIBE DETAIL {CATALOG}.{SCHEMA}.{USER}_sensor_unoptimized
""").select("numFiles", "sizeInBytes").collect()[0]

files_before = detail_before['numFiles']
size_mb = detail_before['sizeInBytes'] / 1024 / 1024

print(f"📊 Before Optimization:")
print(f"   Files: {files_before}")
print(f"   Size: {size_mb:.2f} MB")
print(f"   Avg file size: {size_mb/files_before:.2f} MB")
print(f"\n   Status: {'🔴 Too many small files!' if files_before > 10 else '🟢 OK'}")


In [0]:
# Run OPTIMIZE to compact files
start = time.time()

spark.sql(f"""
OPTIMIZE {CATALOG}.{SCHEMA}.{USER}_sensor_unoptimized
""")

optimize_time = time.time() - start

print(f"✅ OPTIMIZE completed in {optimize_time:.2f} seconds")


In [0]:
# Check results after optimization
detail_after = spark.sql(f"""
DESCRIBE DETAIL {CATALOG}.{SCHEMA}.{USER}_sensor_unoptimized
""").select("numFiles").collect()[0]

files_after = detail_after['numFiles']

print(f"📊 After Optimization:")
print(f"   Files: {files_after}")
print(f"   Reduction: {files_before - files_after} files removed")
print(f"   Improvement: {files_before/files_after:.1f}x fewer files!")
print(f"\n💡 Queries now have much less file I/O overhead!")


## Part 6: Table Optimization - Z-Ordering

**Z-Ordering** organizes data so similar values are stored together.

### How Data Skipping Works:

**Without Z-Ordering:**
```
File 1: devices 1,5,10,15,20     <- Must read
File 2: devices 2,3,8,12,19      <- Must read  
File 3: devices 4,7,9,11,14      <- Must read
```
Query for device_id = 5 must read ALL files!

**With Z-Ordering on device_id:**
```
File 1: devices 1,2,3,4,5        <- Read this
File 2: devices 7,8,9,10,11      <- SKIP
File 3: devices 12,14,15,19,20   <- SKIP
```
Query for device_id = 5 only reads File 1!

### Choosing Z-Order Columns:

✅ **Good candidates:**
- High cardinality (device_id, timestamp)
- Frequently in WHERE clauses
- Used in joins

❌ **Bad candidates:**
- Low cardinality (status: active/inactive)
- Rarely filtered
- Already partitioned by

### Rule: 2-4 columns maximum


# Benchmark query BEFORE Z-Ordering


In [0]:
start = time.time()

result_before_zorder = spark.sql(f"""
SELECT 
    device_id,
    DATE(timestamp) as date,
    AVG(temperature) as avg_temp,
    AVG(rotation_speed) as avg_rotation,
    MAX(air_pressure) as max_pressure,
    COUNT(*) as reading_count
FROM {CATALOG}.{SCHEMA}.{USER}_sensor_unoptimized
WHERE device_id IN (1, 2, 3, 4, 5, 6, 7, 8, 9, 10)
  AND timestamp >= CURRENT_DATE() - INTERVAL 30 DAYS
GROUP BY device_id, DATE(timestamp)
ORDER BY date DESC, device_id
""")

count_before = result_before_zorder.count()
time_before_zorder = time.time() - start

print(f"⏱️  Query Time (Before Z-Order): {time_before_zorder:.2f} seconds")
print(f"   Must scan all files for relevant devices")


In [0]:
# Common dashboard query: Hourly device metrics by factory
# Without materialized view - runs every time

start = time.time()

result_no_mv = spark.sql(f"""
SELECT 
    f.factory_id,
    f.factory_name,
    f.region,
    s.device_id,
    DATE_TRUNC('hour', s.timestamp) as hour,
    AVG(s.temperature) as avg_temp,
    AVG(s.rotation_speed) as avg_rotation,
    AVG(s.air_pressure) as avg_pressure,
    COUNT(*) as reading_count
FROM {CATALOG}.{SCHEMA}.sensor_bronze s
JOIN {CATALOG}.{SCHEMA}.dim_factories f ON s.factory_id = f.factory_id
WHERE s.timestamp >= current_date() - INTERVAL 7 DAYS
GROUP BY f.factory_id, f.factory_name, f.region, s.device_id, DATE_TRUNC('hour', s.timestamp)
ORDER BY hour DESC
LIMIT 100
""")

display(result_no_mv)

no_mv_time = time.time() - start
print(f"\n⏱️  Query time (no materialized view): {no_mv_time:.2f} seconds")


In [0]:
# Create materialized view for this common pattern
spark.sql(f"DROP MATERIALIZED VIEW IF EXISTS {CATALOG}.{SCHEMA}.mv_hourly_factory_metrics")

spark.sql(f"""
CREATE MATERIALIZED VIEW {CATALOG}.{SCHEMA}.mv_hourly_factory_metrics
AS
SELECT 
    f.factory_id,
    f.factory_name,
    f.region,
    s.device_id,
    DATE_TRUNC('hour', s.timestamp) as hour,
    AVG(s.temperature) as avg_temp,
    AVG(s.rotation_speed) as avg_rotation,
    AVG(s.air_pressure) as avg_pressure,
    COUNT(*) as reading_count
FROM {CATALOG}.{SCHEMA}.sensor_bronze s
JOIN {CATALOG}.{SCHEMA}.dim_factories f ON s.factory_id = f.factory_id
GROUP BY f.factory_id, f.factory_name, f.region, s.device_id, DATE_TRUNC('hour', s.timestamp)
""")

print("✅ Created materialized view")
print("   This pre-computes the expensive join and aggregation")


In [0]:
# Now query is MUCH faster - reads pre-computed results
start = time.time()

result_with_mv = spark.sql(f"""
SELECT *
FROM {CATALOG}.{SCHEMA}.mv_hourly_factory_metrics
WHERE hour >= current_date() - INTERVAL 7 DAYS
ORDER BY hour DESC
LIMIT 100
""")

display(result_with_mv)

mv_time = time.time() - start
mv_speedup = no_mv_time / mv_time if mv_time > 0 else 0

print(f"\n⏱️  Query time (with materialized view): {mv_time:.2f} seconds")
print(f"🚀 Speedup: {mv_speedup:.1f}x faster!")
print(f"\n💡 Dashboard loads instantly instead of making users wait!")


### 🎯 Materialized View Benefits:

1. **Dashboard speed**: Instant load times
2. **Cost savings**: Compute once, query many times
3. **Automatic refresh**: Stays up to date
4. **Query rewriting**: Optimizer uses it automatically

**For your deadline:** This makes your real-time dashboard actually real-time!


## 7. Caching Strategies <a id="caching"></a>

**Caching** keeps frequently accessed data in memory for instant access.

### Types of Caching:

1. **DataFrame Cache**: Temporary, session-specific
2. **Delta Cache**: Disk-based, persists across queries
3. **Result Cache**: Caches query results

### When to Use Caching:

✅ Dimension tables (small, frequently joined)  
✅ Reference data  
✅ Iterative ML training  
✅ Dashboard data sources  

❌ Don't cache:
- Large fact tables (waste of memory)
- Rarely accessed data
- Data that changes frequently


In [0]:
# Cache frequently used dimension tables
# These are joined in almost every query!

spark.sql(f"CACHE TABLE {CATALOG}.{SCHEMA}.dim_factories")
spark.sql(f"CACHE TABLE {CATALOG}.{SCHEMA}.dim_models")
spark.sql(f"CACHE TABLE {CATALOG}.{SCHEMA}.dim_devices")

print("✅ Cached dimension tables")
print("   Joins with these tables are now instant!")


In [0]:
# Test query with cached dimensions
start = time.time()

result_cached = spark.sql(f"""
SELECT 
    f.factory_name,
    f.region,
    m.model_name,
    m.model_family,
    d.device_id,
    COUNT(DISTINCT s.trip_id) as trip_count,
    AVG(s.temperature) as avg_temp
FROM {CATALOG}.{SCHEMA}.sensor_bronze s
JOIN {CATALOG}.{SCHEMA}.dim_devices d ON s.device_id = d.device_id
JOIN {CATALOG}.{SCHEMA}.dim_factories f ON d.factory_id = f.factory_id
JOIN {CATALOG}.{SCHEMA}.dim_models m ON d.model_id = m.model_id
WHERE s.timestamp >= current_date() - INTERVAL 1 DAYS
GROUP BY f.factory_name, f.region, m.model_name, m.model_family, d.device_id
""")

display(result_cached)

cached_time = time.time() - start
print(f"\n⏱️  Query time (with cached dimensions): {cached_time:.2f} seconds")
print("✨ Dimension joins are instant - no disk I/O needed!")


In [0]:
# Clear cache when done (frees memory)
spark.sql(f"UNCACHE TABLE IF EXISTS {CATALOG}.{SCHEMA}.dim_factories")
spark.sql(f"UNCACHE TABLE IF EXISTS {CATALOG}.{SCHEMA}.dim_models")
spark.sql(f"UNCACHE TABLE IF EXISTS {CATALOG}.{SCHEMA}.dim_devices")

print("✅ Cleared caches")


### 💡 Caching Best Practices:

1. **Cache small tables** that are joined frequently
2. **Monitor memory** - don't cache everything
3. **Clear caches** when not needed
4. **Use Delta cache** on read-heavy clusters
5. **Let Databricks auto-cache** query results


## 8. Performance Comparison Summary <a id="comparison"></a>

Let's summarize the performance improvements:

### Optimization Results:

| Technique | Typical Speedup | Setup Time | Maintenance |
|-----------|----------------|------------|--------------|
| **File Compaction** | 2-3x | 5 min | Weekly |
| **Z-Ordering** | 3-10x | 10 min | Weekly |
| **Liquid Clustering** | 3-10x | 15 min | Automatic |
| **Materialized Views** | 5-20x | 15 min | Automatic |
| **Caching** | 10-100x | 2 min | Per session |

### Impact on Your Project:

**Before Optimization:**
- Dashboard: 10-15 seconds to load ❌
- Model training queries: 5 minutes ❌
- Ad-hoc analysis: 30+ seconds ❌
- Leadership impatient: Yes ❌

**After Optimization:**
- Dashboard: <1 second ✅
- Model training queries: 30 seconds ✅
- Ad-hoc analysis: 3-5 seconds ✅
- Leadership happy: Yes! ✅

### Optimization Strategy:

1. **Start with Z-Ordering** - Quick wins on existing tables
2. **Add Materialized Views** - For dashboard queries
3. **Cache dimensions** - Small tables used everywhere
4. **Use Liquid Clustering** - For new production tables
5. **Monitor and adjust** - Query history shows what needs work


In [0]:
# Quick performance audit of your tables
print("📊 Table Performance Audit\n")

# Get table details from sensor tables
tables_to_check = ['sensor_bronze', 'sensor_unoptimized', 'sensor_clustered']

for table in tables_to_check:
    try:
        detail = spark.sql(f"DESCRIBE DETAIL {CATALOG}.{SCHEMA}.{table}").collect()[0]
        num_files = detail['numFiles']
        size_mb = detail['sizeInBytes'] / 1024 / 1024
        
        if num_files > 1000:
            rec = '🔴 Too many files - run OPTIMIZE'
        elif num_files > 100:
            rec = '🟡 Consider OPTIMIZE'
        else:
            rec = '🟢 File count OK'
        
        print(f"{table}:")
        print(f"  Size: {size_mb:.2f} MB")
        print(f"  Files: {num_files}")
        print(f"  {rec}\n")
    except:
        print(f"{table}: Table not found or error\n")

print("💡 Use this audit to identify tables needing optimization")


## 🎯 Key Takeaways

### Must-Do Optimizations:

1. **Z-Order your fact tables** - `OPTIMIZE table ZORDER BY (common_filters)`
2. **Create materialized views** - For repeated dashboard queries
3. **Cache dimension tables** - Small, frequently joined tables
4. **Monitor file count** - Run OPTIMIZE when >100 files
5. **Use Liquid Clustering** - For new production tables

### Performance Checklist:

- [ ] Z-Ordered sensor_bronze on (device_id, timestamp)
- [ ] Created materialized views for dashboard queries
- [ ] Cached dimension tables
- [ ] Compacted files (numFiles < 100)
- [ ] Enabled Photon on SQL warehouse
- [ ] Reviewed slow queries in Query History

### For Your End-of-Week Demo:

✅ **Dashboards**: Sub-second response times  
✅ **ML models**: Fast training on optimized data  
✅ **Genie queries**: Instant results on materialized views  
✅ **Leadership**: Impressed with performance  

---

## 🚀 Try This Out

### Challenge 1: Optimize Your Most Expensive Query

1. Check SQL warehouse Query History
2. Find the slowest query from yesterday
3. Apply Z-Ordering or create materialized view
4. Measure the speedup

### Challenge 2: Create a Performance Dashboard

Build a dashboard that tracks:
- Table file counts
- Query execution times
- Cache hit rates
- Top slow queries

### Challenge 3: Optimize the Inspection Pipeline

1. Z-Order `inspection_bronze` on (device_id, timestamp)
2. Create materialized view for defect rate by model
3. Compare query performance before/after

### Challenge 4: Liquid Clustering Experiment

1. Create a new table with Liquid Clustering
2. Try different clustering keys
3. Compare performance vs Z-Ordering

---

**Next Steps:**
- Apply these techniques to your production tables
- Set up monitoring to track query performance
- Schedule weekly OPTIMIZE jobs
- Educate team on performance best practices

**Remember:** Fast queries = happy leadership = successful project! 🎉


In [0]:
# Uncomment to clean up demo tables
# spark.sql(f"DROP TABLE IF EXISTS {CATALOG}.{SCHEMA}.{USER}_sensor_unoptimized")
# spark.sql(f"DROP TABLE IF EXISTS {CATALOG}.{SCHEMA}.{USER}_sensor_clustered")
# spark.sql(f"DROP MATERIALIZED VIEW IF EXISTS {CATALOG}.{SCHEMA}.{USER}_mv_hourly_metrics")
# print("✅ Cleaned up demo tables")


### 🔍 Query Plan Analysis

Look for these indicators in the plan:
- **FileScan**: How many files are scanned?
- **Filter**: Pushed down to file scan (good) or after (bad)?
- **Exchange**: Data shuffle between nodes (expensive)
- **Data Skipping**: Are file statistics used?

**Key Issue:** Without optimization, Databricks must scan ALL files even though we only need 5 devices!
